# CIFAR-10: Data Augmentation y Arquitecturas Avanzadas

En esta notebook, exploraremos técnicas de **data augmentation** y el uso de **arquitecturas avanzadas** como DenseNet para mejorar el rendimiento de modelos de clasificación en el dataset CIFAR-10. Abordaremos también los desafíos de las redes CNN tradicionales y cómo las arquitecturas modernas pueden superarlos.

## Introducción

### Objetivos

1. **Implementar y entender técnicas de data augmentation** para aumentar la diversidad del conjunto de entrenamiento y mejorar la generalización de los modelos.
2. **Comparar arquitecturas de redes neuronales** tradicionales con arquitecturas más avanzadas como DenseNet.
3. **Entrenar y evaluar modelos de clasificación** en PyTorch utilizando el dataset CIFAR-10.
4. **Analizar el impacto de las técnicas de data augmentation y las arquitecturas avanzadas** en el rendimiento del modelo utilizando métricas adecuadas.

### Contenido

1. Configuración de bibliotecas y semillas para reproducibilidad.
2. Familiarización con `torchvision.transforms`.
2. Carga y exploración del dataset CIFAR-10.
3. Aplicación de técnicas de data augmentation.
4. Implementación y comparación de modelos CNN tradicionales y DenseNet.
5. Evaluación y comparación del rendimiento de los modelos con y sin data augmentation y utilizando diferentes arquitecturas.

### Sobre el conjunto de datos

El dataset CIFAR-10 es un conjunto de datos muy utilizado en la investigación de la visión por computadora. Contiene 60,000 imágenes de 32x32 píxeles en 10 clases diferentes, con 6,000 imágenes por clase. Este conjunto de datos es ideal para entrenar y evaluar modelos de clasificación, permitiendo explorar la efectividad de diversas técnicas de aumento de datos y arquitecturas avanzadas.

El modelo está disponible en el siguiente [link](https://www.cs.toronto.edu/~kriz/cifar.html), pero PyTorch proporciona una interfaz para descargar y cargar el conjunto de datos de forma sencilla.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import v2 as T
from torchvision.io import read_image

from torchinfo import summary

import os
from pathlib import Path
import numpy as np

from utils import (
    train,
    plot_training,
    model_classification_report,
    show_tensor_image,
    show_tensor_images,
)

In [ ]:
# Fijamos la semilla para que los resultados sean reproducibles
SEED = 23

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import sys

# definimos el dispositivo que vamos a usar
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS
elif torch.xpu.is_available():
    DEVICE = "xpu"  # si no hay GPU, pero hay XPU, usamos XPU

print(f"Usando {DEVICE}")

NUM_WORKERS = 0 # Win y MacOS pueden tener problemas con múltiples workers
if sys.platform == 'linux':
    NUM_WORKERS = 4  # numero de workers para cargar los datos (depende de cada caso)

print(f"Usando {NUM_WORKERS}")

In [ ]:
BATCH_SIZE = 512  # tamaño del batch

## Data Augmentation

El **data augmentation** es una técnica de **regularización** que consiste en aplicar transformaciones aleatorias a las imágenes de entrenamiento, como rotaciones, traslaciones, reflejos, recortes, etc. Esto aumenta la diversidad del conjunto de entrenamiento y ayuda a mejorar la generalización del modelo.

PyTorch proporciona una interfaz sencilla para aplicar data augmentation a través de la clase [`torchvision.transforms`](https://pytorch.org/vision/main/transforms.html). Primero tomemos una imagen de ejemplo y apliquemos algunas transformaciones comunes. 

In [ ]:
img = read_image(str(Path("assets") / "dwight.jpg"))
show_tensor_image(img, title="Original Image")

Las transformaciones más comunes incluyen:

- [`RandomHorizontalFlip`](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.RandomHorizontalFlip.html): Voltea horizontalmente la imagen con una probabilidad dada.
- [`RandomCrop`](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.RandomCrop.html): Recorta la imagen de forma aleatoria.
- [`RandomRotation`](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.RandomRotation.html): Rota la imagen de forma aleatoria.
- [`ColorJitter`](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.ColorJitter.html): Cambia el brillo, contraste, saturación y tono de la imagen de forma aleatoria.
- [`RandomResizedCrop`](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.RandomResizedCrop.html): Recorta y cambia el tamaño de la imagen de forma aleatoria.
- ...

Se puede ver la lista completa de transformaciones en la [documentación oficial](https://pytorch.org/vision/stable/transforms.html#v2-api-reference-recommended).

In [ ]:
img_rhf = T.RandomHorizontalFlip(p=1)(img)  # p=1 para que siempre se aplique
img_rc = T.RandomCrop(size=(100, 100))(img)  # tomamos un recorte de 100x100
img_rr = T.RandomRotation(degrees=45)(
    img
)  # rotamos la imagen de forma aleatoria entre -45 y 45 grados
img_cj = T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)(
    img
)  # aplicamos un cambio de color
img_rrc = T.RandomResizedCrop(size=(200, 200))(
    img
)  # tomamos un recorte y redimensionamos a 200x200

show_tensor_images(
    [img, img_rhf, img_rc, img_rr, img_cj, img_rrc],
    titles=[
        f"original ({img.shape[-2]}x{img.shape[-1]})",
        f"HorizontalFlip ({img_rhf.shape[-2]}x{img_rhf.shape[-1]})",
        f"Crop ({img_rc.shape[-2]}x{img_rc.shape[-1]})",
        f"Rotation ({img_rr.shape[-2]}x{img_rr.shape[-1]})",
        f"ColorJitter ({img_cj.shape[-2]}x{img_cj.shape[-1]})",
        f"ResizedCrop ({img_rrc.shape[-2]}x{img_rrc.shape[-1]})",
    ],
)

Tamibén se pueden combinar transformaciones utilizando [`Compose`](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Compose).

In [ ]:
composed = T.Compose([T.RandomHorizontalFlip(p=1), T.CenterCrop(size=(25, 100))])

img_composed = composed(img)
show_tensor_image(img_composed)

En general vamos a aplicar las transformaciones a las imágenes de entrenamiento, pero no a las de validación y test. Por lo tanto, es importante tener dos pipelines de transformaciones, uno para entrenamiento y otro para validación/test.

> Es importante considerar que no todas las transformaciones son adecuadas para todas las tareas. Por ejemplo, en el caso del dataset MNIST, no se debería aplicar `RandomHorizontalFlip` ya que los dígitos perderían su significado al voltearse horizontalmente. De manera similar, usar `RandomVerticalFlip` podría causar problemas con dígitos como el 6 y el 9, que se confundirían al invertirse verticalmente.


## Dataset y DataLoader


### Dataset

El dataset [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) es un conjunto de datos ampliamente utilizado en el campo del aprendizaje automático y la visión por computadora. Contiene 60,000 imágenes a color divididas en 10 clases, tales como aviones, automóviles, pájaros, gatos y perros. Cada imagen es de 32x32 píxeles, lo que lo hace ideal para probar algoritmos de clasificación de imágenes.

Para facilitar el manejo del dataset CIFAR-10, utilizaremos la biblioteca `torchvision.datasets`, la cual proporciona una manera sencilla de cargar y preprocesar estos datos.

### DataLoader

Para facilitar los experimentos y el entrenamiento de modelos vamos a definir una función `get_dataloaders` que nos devolverá los DataLoaders para los conjuntos de entrenamiento, validación y test con las transformaciones deseadas.

In [ ]:
DATA_DIR = Path("data")

# Wrapper para aplicar transformaciones
class TransformDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform
    
    def __len__(self):
        return len(self.subset)
    
    def __getitem__(self, idx):
        image, label = self.subset[idx]
        return self.transform(image), label

def get_dataloaders(
    train_transf, test_transf, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS
):
    """
    Función para obtener los dataloaders de entrenamiento, validación y test

    Args:
    - train_transf: transformaciones para el dataset de entrenaiento
    - test_transf: transformaciones para el dataset de test
    - batch_size: tamaño del batch
    - num_workers: número de workers para cargar los datos
    """

    # descargamos el dataset CIFAR10 (si no lo tenemos ya)
    train_dataset = datasets.CIFAR10(
        DATA_DIR, train=True, download=True, transform=None
    )
    test_dataset = datasets.CIFAR10(
        DATA_DIR, train=False, download=True, transform=test_transf
    )

    # dividimos el dataset de entrenamiento en entrenamiento y validación
    train_size = int(0.8 * len(train_dataset))
    valid_size = len(train_dataset) - train_size
    train_dataset, validation_dataset = random_split(
        train_dataset,
        [train_size, valid_size],
        generator=torch.Generator().manual_seed(SEED),  # fijamos la semilla
    )

    # aplicamos las transformaciones
    train_dataset = TransformDataset(train_dataset, train_transf)
    validation_dataset = TransformDataset(validation_dataset, test_transf)

    # creamos los dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    valid_loader = DataLoader(
        validation_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    return train_loader, valid_loader, test_loader

In [ ]:
NUM_CLASES = 10  # número de clases en CIFAR10

### Transformaciones

Vamos a definir 3 pipelines de transformaciones:

1. **Transformaciones sin data augmentation**: Estas transformaciones se aplican al conjunto de entrenamiento sin utilizar técnicas de data augmentation. Esto nos proporciona un punto de referencia sobre el rendimiento del modelo con datos sin alterar.
2. **Transformaciones con data augmentation**: Estas transformaciones se aplican al conjunto de entrenamiento utilizando técnicas de data augmentation. La data augmentation ayuda a aumentar la variabilidad de los datos de entrenamiento, mejorando la capacidad del modelo para generalizar.

In [ ]:
train_dataset = datasets.CIFAR10(
    DATA_DIR, train=True, download=True, transform=T.Compose([
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
    ])
)

# Stack todas las imágenes en un tensor
data = torch.stack([img for img, _ in train_dataset])
# Shape: (50000, 3, 32, 32)

# Calcula mean y std por canal
T_MEAN = data.mean(dim=[0, 2, 3])  # Promedia sobre samples, height, width
T_STD = data.std(dim=[0, 2, 3])

print(f"Mean: {T_MEAN}")
print(f"Std: {T_STD}")

In [ ]:
# Transformaciones con data augmentation
train_transforms_aug = T.Compose(
    [
        # Data augmentation:
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(degrees=10),
        # Transformaciones comunes:
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
        T.Normalize(mean=T_MEAN, std=T_STD),
    ]
)

# Transformaciones con data augmentation
train_transforms_no_aug = T.Compose(
    [
        # Transformaciones comunes:
        T.ToImage(),
        T.ToDtype(torch.float32, scale=True),
        T.Normalize(mean=T_MEAN, std=T_STD),
    ]
)


# Transformaciones sin data augmentation
test_transform = T.Compose(
    [
        T.ToImage(),  # tenemos que convertir la imagen PIL a tensor
        T.ToDtype(torch.float32, scale=True),
        T.Normalize(mean=T_MEAN, std=T_STD),
    ]
)

## Parte 1: Data Augmentation

En este experimento, compararemos el rendimiento de un modelo de clasificación de imágenes en el dataset CIFAR-10 con y sin data augmentation. Utilizaremos una arquitectura de red neuronal convolucional tradicional y evaluaremos el rendimiento en términos de precisión y pérdida.

### Modelo base: LeNet

Tomemos [LeNet](https://d2l.ai/chapter_convolutional-neural-networks/lenet.html) para este experimento. LeNet es una red neuronal convolucional simple y nos permitirá comparar el impacto de la data augmentation en un modelo básico.


In [ ]:
class LeNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(LeNet, self).__init__()
        self.c1 = nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5)
        self.s2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.s4 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.f6 = nn.Linear(in_features=120, out_features=84)
        self.output = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.tanh(self.c1(x))
        x = self.s2(x)
        x = F.tanh(self.c3(x))
        x = self.s4(x)
        x = F.tanh(self.c5(x))
        x = x.flatten(start_dim=1)
        x = F.tanh(self.f6(x))
        x = self.output(x)
        return x

summary(LeNet(3, NUM_CLASES), input_size=(BATCH_SIZE, 3, 32, 32))

In [ ]:
# definicion de hiperparametros
LR = 0.001
EPOCHS = 30
# definimos
CRITERION = nn.CrossEntropyLoss().to(DEVICE)

### Entrenamiento sin data augmentation

In [ ]:
train_loader, val_loader, test_loader = get_dataloaders(
    train_transforms_no_aug, test_transform
)

In [ ]:
letnet_model = LeNet(3, NUM_CLASES).to(DEVICE)
optimizer = optim.Adam(letnet_model.parameters(), lr=LR)

In [ ]:
train_errors_ln_nda, val_errors_ln_nda = train(
    model=letnet_model,
    optimizer=optimizer,
    criterion=CRITERION,
    train_loader=train_loader,
    val_loader=val_loader,
    device=DEVICE,
    do_early_stopping=True,
    patience=3,
    epochs=EPOCHS,
)

plot_training(train_errors_ln_nda, val_errors_ln_nda)

In [ ]:
model_classification_report(letnet_model, test_loader, DEVICE, NUM_CLASES)

### Entrenamiento con data augmentation

In [ ]:
letnet_model = LeNet(3, NUM_CLASES).to(DEVICE)
optimizer = optim.Adam(letnet_model.parameters(), lr=LR)

train_loader, val_loader, test_loader = get_dataloaders(
    train_transforms_aug, test_transform
)

In [ ]:
train_errors_ln_da, val_errors_ln_da = train(
    model=letnet_model,
    optimizer=optimizer,
    criterion=CRITERION,
    train_loader=train_loader,
    val_loader=val_loader,
    device=DEVICE,
    do_early_stopping=True,
    patience=3,
    epochs=EPOCHS,
)

plot_training(train_errors_ln_da, val_errors_ln_da)

In [ ]:
model_classification_report(letnet_model, test_loader, DEVICE, NUM_CLASES)

### Ejercicios

1. ¿Qué mejoras observas al utilizar data augmentation en este experimento?
2. Experimenta con diferentes transformaciones y evalúa su impacto en el rendimiento del modelo.

In [ ]:
# TODO Ej1

In [ ]:
# TODO Ej2

## Parte 2: DenseNet

[DenseNet](https://d2l.ai/chapter_convolutional-modern/densenet.html) es una arquitectura de red neuronal convolucional que aborda los problemas de las redes tradicionales conectando cada capa con todas las capas subsiguientes. Esto permite que la información fluya de manera más eficiente a través de la red y facilita el entrenamiento de redes profundas.

Se recomienda leer los articulos:
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) 
- [Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993)

### Arquitectura DenseNet

Podemos dividir DenseNet en cuatro partes principales:
- **Convolucion inicial**: Una sola capa convolucional seguida de una capa de pooling.
- **Bloques densos**: Varios bloques densos, cada uno compuesto por múltiples capas convolucionales conectadas entre sí.
- **Transiciones**: Capas convolucionales que reducen la altura y el ancho de las activaciones.
- **Capa de salida**: Una capa de pooling global y una capa completamente conectada.

![Image](https://miro.medium.com/max/5164/1*_Y7-f9GpV7F93siM1js0cg.jpeg)

#### Bloque denso

Los [bloques densos](https://d2l.ai/chapter_convolutional-modern/densenet.html#dense-blocks) son la característica principal de DenseNet. Cada bloque denso conecta cada capa con todas las capas subsiguientes en un patrón denso (de ahí el nombre). Esto permite que la información fluya de manera más eficiente a través de la red y facilita el entrenamiento de redes profundas.

Los bloques densos están compuestos por capas densa que constan de:
- Una capa convolucional.
- Un capa de Batch Normalization.
- Una función de activación ReLU.

Luego en el forward se concatenan la salida con la entrada usando la función [`torch.cat`](https://pytorch.org/docs/stable/generated/torch.cat.html).

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-20_at_11.33.17_PM_Mt0HOZL.png" width="500">

La forma de controlar la cantidad de canales en cada capa es mediante el parámetro `growth_rate`, que indica cuántos canales se agregarán en cada capa convolucional dentro de un bloque denso.

In [ ]:
class DenseLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DenseLayer, self).__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
in_channels  = 6  # número de canales de entrada
out_channels = 4 # número de canales de salida
dense_layer = DenseLayer(in_channels=in_channels, out_channels=out_channels)
summary(dense_layer, input_size=(BATCH_SIZE, in_channels, 32, 32))

In [ ]:
class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(DenseBlock, self).__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
in_channels = 6  # número de canales de entrada
num_layer   = 5  # número de capas en el bloque denso
growth_rate = 4  # tasa de crecimiento

dense_block = DenseBlock(num_layer, in_channels, growth_rate)

summary(dense_block, input_size=(BATCH_SIZE, in_channels, 32, 32))

#### Transiciones

Los bloques densos pueden generar una gran cantidad de canales, lo que puede aumentar significativamente la complejidad del modelo. Para reducir la cantidad de canales y controlar la complejidad del modelo, DenseNet utiliza [capas de transición](https://d2l.ai/chapter_convolutional-modern/densenet.html#transition-layers) entre los bloques densos. Estas capas consisten en una capa convolucional seguida de una capa de pooling.

Por lo cual:
- Reducen la cantidad de canales a través de una capa convolucional.
- Reducen la altura y el ancho de las activaciones (downsampling).

In [ ]:
class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
in_channels = 100  # número de canales de entrada
out_channels = 25  # número de canales de salida (en general, se reduce a la mitad)
img_w = 32  # ancho de la imagen
img_h = 32  # alto de la imagen

transition_layer = TransitionLayer(in_channels, out_channels)

summary(transition_layer, input_size=(BATCH_SIZE, in_channels, img_w, img_h))

#### DenseNet Model


In [ ]:
class DenseNet(nn.Module):
    def __init__(self, num_layers = [6, 12, 24, 16], growth_rate=32, num_classes=10):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate
        self.num_layers = num_layers
        self.num_blocks = len(num_layers)

        # Capa inicial: se encarga de extraer características iniciales

        # Bloques densos y capas de transición en nn.Sequential
        
        # Capa de clasificación
        
        pass

    def forward(self, x):
        pass


# Ejemplo de uso
model = DenseNet(num_layers=[6, 12, 24, 16], growth_rate=32, num_classes=10)

summary(model, input_size=(BATCH_SIZE, 3, 32, 32), depth=3)

In [ ]:
model = DenseNet(
    num_layers=[6, 12, 24, 16], growth_rate=12, num_classes=10
).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=LR)

train_errors_dn, val_errors_dn = train(
    model=model,
    optimizer=optimizer,
    criterion=CRITERION,
    train_loader=train_loader,
    val_loader=val_loader,
    device=DEVICE,
    do_early_stopping=True,
    patience=3,
    epochs=EPOCHS,
)

In [ ]:
plot_training(train_errors_dn, val_errors_dn)

In [ ]:
model_classification_report(model, test_loader, DEVICE, NUM_CLASES)

### Ejercicios

1. Utilizar otras transformaciones para data augmentation y evaluar su impacto en el rendimiento del modelo.
2. Experimentar con diferentes números de bloques densos y growth rates en DenseNet.
3. Utilizar Dropout y evaluar su impacto en el rendimiento del modelo.